In [ ]:
class Convolution:
    def __init__(self, input_shape, filter_size, num_filters):
        input_height, input_width = input_shape
        self.num_filters = num_filters
        self.input_shape = input_shape
        # Size of outputs and filters or kernels or weights
        self.filter_size = filter_size
        self.filter_shape = (num_filters, filter_size, filter_size)
        self.output_shape = (num_filters, input_height - filter_size + 1, input_width - filter_size + 1)

        # Initialize filters and biases
        self.filters = self.random_normal(self.filter_shape)
        self.biases = self.random_normal(self.output_shape)
        #print("shape0: ", self.filter_shape, self.filters.shape)

    def random_normal(self, shape):
        import random
        matrix = []
        for _ in range(shape[0]):
            row = []
            for _ in range(shape[1]):
                column = []
                for _ in range(shape[2]):
                    value = 0.1 * (2 * random.random() - 1)
                    column.append(value)
                row.append(column)
            matrix.append(row)
        return matrix

    def forward(self, input_data):
        self.input_data = input_data
        # Initialize the input value
        #print("output shape: ", self.output_shape, len(self.output_shape), self.output_shape[0])
        output = self.zeros(self.output_shape)
        for i in range(self.num_filters):
            #print("filtershape forward: ", self.filters[i].shape)
            output[i] = self.correlate2d(input_data, self.filters[i], self.filter_size, mode="valid")
            # Applying Relu Activation function
            output[i] = self.maximum(output[i], 0, self.output_shape[1])
        return output

    def backward(self, dL_dout, lr):
        # Create a random dL_dout array to accommodate output gradients
        dL_dinput = self.zeros(self.input_shape)
        dL_dfilters = self.zeros(self.filter_shape)

        for i in range(self.num_filters):
            # Calculating the gradient of loss with respect to kernels
            dL_dfilters[i] = self.correlate2d2(self.input_data, dL_dout[i], self.filter_size, mode="valid")

            # Calculating the gradient of loss with respect to inputs
            #dL_dinput += self.correlate2d(dL_dout[i], self.filters[i], mode="full")

            #print("weights of filters:", self.filters)
            # Updating the parameters with learning rate
            self.filters = self.update(self.filters, dL_dfilters[i], lr)
            #self.biases = self.update(self.biases, dL_dout[i], lr)
            #print("weights of filters:", self.filters)

        # Returning the gradient of inputs
        return dL_dinput

    def correlate2d2(self, input_data, kernel, size, mode):
        #print("shape1: ", input_data.shape)
        #print("klernel: ", len(kernel), len(kernel[0]))
        #print("inputdtat_ shape: ", input_data.shape[0] - size + 1, input_data.shape[1] - size + 1)
        output = self.zeros((size, size))
        output1 = self.zeros((size, size))
        #print("ouyput cor: ", output)
        #print("HIIIIIII")
        for i in range(input_data.shape[0] - size + 1):
            for j in range(input_data.shape[1] - size + 1):
                output = self.multiply1( input_data[i:i+size, j:j+size], kernel[i][j])
                #print("covnpotputy:", output)
                for m in range(size):
                  for n in range(size):
                    output1[m][n] += output[m][n]
        #print("convoutput:", output1)
        return output1

    def multiply1(self, mat1, scal):
      output1 = self.zeros((self.filter_size, self.filter_size))
      for i in range(len(mat1)):
        for j in range(len(mat1[0])):
          output1[i][j] = mat1[i][j] * scal
      return output1

    def correlate2d(self, input_data, kernel, size, mode):
        #print("shape1: ", input_data.shape)
        #print("shape2: ", kernel.shape)
        #print("inputdtat_ shape: ", input_data.shape[0] - size + 1, input_data.shape[1] - size + 1)
        output = self.zeros((input_data.shape[0] - size + 1, input_data.shape[1] - size + 1))
        #print("ouyput cor: ", output)
        #print("HIIIIIII")
        for i in range(input_data.shape[0] - size + 1):
            for j in range(input_data.shape[1] - size + 1):
                output[i][j] = self.sum(self.multiply(input_data[i:i+size, j:j+size], kernel), size)
        return output

    def maximum(self, arr, val, size):
        #print("maximum!!!")
        #print(size)
        output = [[0 for _ in range(size)] for _ in range(size)]
        for i in range(size):
            for j in range(size):
                output[i][j] = max(arr[i][j], val)
        return output

    def multiply(self, arr1, arr2):
        #print("multiply!!!")
        #print("multi shape: ", arr1.shape, len(arr1), len(arr1.shape), arr1)
        output = self.zeros(arr1.shape)
        for i in range(arr1.shape[0]):
            for j in range(arr1.shape[1]):
                output[i][j] = arr1[i][j] * arr2[i][j]
        return output

    def sum(self, arr, size):
        output = 0
        for i in range(size):
            for j in range(size):
                output += arr[i][j]
        return output

    def update(self, arr, grad, lr):
        #output = self.zeros(arr.shape)
            for j in range(len(arr)):
                for k in range(len(arr[0])):
                    arr[j][k] = arr[j][k] - lr * grad[j][k]
            return arr

    def zeros(self, shape):
        #print( shape[1], shape[0], len(shape))
        if len(shape)<3:
          return [[0 for _ in range(shape[1])] for _ in range(shape[0])]
        else:
          return [[[0 for _ in range(shape[2])] for _ in range(shape[1])] for _ in range(shape[0])]

In [ ]:
class MaxPool:
    def __init__(self, pool_size, kersize, input_hight, input_width):
        self.pool_size = pool_size
        self.num_channels = kersize
        self.input_height = input_hight
        self.input_width = input_width
    def forward(self, input_data):
        self.input_data = input_data
        self.output_height = self.input_height // self.pool_size
        self.output_width = self.input_width // self.pool_size

        # Determining the output shape
        self.output = self.zeros((self.num_channels, self.output_height, self.output_width))

        # Looping through different channels
        for c in range(self.num_channels):
            # Looping through the height
            for i in range(self.output_height):
                # Looping through the width
                for j in range(self.output_width):
                    # Starting position
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    # Ending Position
                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size

                    # Creating a patch from the input data
                    patch = self.get_patch(input_data, c, start_i, end_i, start_j, end_j)

                    # Finding the maximum value from each patch/window
                    #print("pool size: ", self.pool_size)
                    self.output[c][i][j] = self.max_value(patch, self.pool_size)

        return self.output

    def backward(self, dL_dout, lr):
        dL_dinput = [[[0 for _ in range(self.input_width)] for _ in range(self.input_height)]for _ in range(self.num_channels)]

        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size
                    patch = self.get_patch(self.input_data, c, start_i, end_i, start_j, end_j)

                    mask = self.create_mask(patch)
                    #print("mask:",mask)
                    for m in range(self.pool_size):
                      for n in range(self.pool_size):
                        mask[m][n] = dL_dout[c][i][j] * mask[m][n]
                        dL_dinput[c][start_i+m][start_j+n] = mask[m][n]

                    #dL_dinput[c][start_i:end_i][start_j:end_j] = mask
                    #print("dl_dinput:", dL_dinput[c][start_i:end_i][start_j:end_j])

        #print(dL_dinput)
        return dL_dinput

    def get_patch(self, input_data, channel, start_i, end_i, start_j, end_j):
        patch = self.zeros((end_i - start_i, end_j - start_j))
        for x in range(start_i, end_i):
            for y in range(start_j, end_j):
                patch[x - start_i][y - start_j] = input_data[channel][x][y]
        return patch

    def max_value(self, patch, patch_size):
        max_val = patch[0][0]
        for x in range(patch_size):
            for y in range(patch_size):
                if patch[x][y] > max_val:
                    max_val = patch[x][y]
        return max_val

    def create_mask(self, patch):
        mask = [[0 for _ in range(self.pool_size)] for _ in range(self.pool_size)]
        max_val = self.max_value(patch, self.pool_size)
        for x in range(self.pool_size):
            for y in range(self.pool_size):
                #print("maskval == patch:", patch[x][y], max_val)
                if patch[x][y] == max_val:
                    mask[x][y] = 1
        return mask

    def zeros(self, shape):
        if len(shape)<3:
          return [[0 for _ in range(shape[1])] for _ in range(shape[0])]
        else:
          return [[[0 for _ in range(shape[2])] for _ in range(shape[1])] for _ in range(shape[0])]

    def zeros_like(self, arr):
        shape = arr.shape
        return self.zeros(shape)

In [ ]:
class Fully_Connected:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = self.random_normal1(output_size, input_size)
        self.biases = self.random_normal2(output_size)

    def random_normal1(self, shape1, sh2):
        import random
        return [[0.1 * (2 * random.random() - 1) for _ in range(sh2)] for _ in range(shape1)]


    def random_normal2(self, shape):
        import random
        return [0.1 * (2 * random.random() - 1) for _ in range(shape)]

    def softmax(self, z):
        exp_values = self.exp_values(z)
        sum_exp_values = sum(exp_values)
        probabilities = [val / sum_exp_values for val in exp_values]
        return probabilities


    def exp_values(self, z):
        e = 2.7183
        exp_values = [e**val for val in z]
        #print(exp_values)
        return exp_values


    def softmax_derivative(self, s):
        s_matrix = [[0 for _ in range(len(s))] for _ in range(len(s))]
        s_dot_st = [[0 for _ in range(len(s))] for _ in range(len(s))]
        subtracted_matrix =  s_dot_st
        for i in range(len(s)):
            s_matrix[i][i] = s[i]

        for i2 in range(len(s)):
          for j in range(len(s)):
            s_dot_st[i2][j] = s[i2] * s[j]

        for z1 in range(len(s_matrix)):
          for z2 in range(len(s_matrix[0])):
            subtracted_matrix[z1][z2] = s_matrix[z1][z2] -  s_dot_st[z1][z2]

        return subtracted_matrix

    def forward(self, input_data):
        self.input_data = input_data
        flattened_input = self.flatten_input(input_data)
        self.z = self.calculate_weighted_sum(flattened_input)
        #print("ZZZZ input of sooftmax:", self.z)
        self.output = self.softmax(self.z)
        #print("output of softmax:", self.output)
        return self.output

    def flatten_input(self, input_data):
        #print("inputdate:", len(input_data), input_data)
        #print("inputShape:", input_data.shape)
        flattened_input = []
        for chanel in input_data:
          for row in chanel:
              for val in row:
                  #print("val:",val)
                  flattened_input.append(val)
        #print("flatten data:", flattened_input)
        return flattened_input

    def calculate_weighted_sum(self, flattened_input):
        weighted_sum = []
        #print("output size:", self.output_size)
        #print("input size:", self.input_size)
        #print("len of weight:", len(self.weights))
        #print("len of flatten:", len(flattened_input))
        #print("weights:", self.weights)
        for i in range(len(self.weights)):
            sum_val = 0
            for j in range(len(flattened_input)):
                #print("i and j:", i,j)
                #print("weight:", self.weights[i])
                #print("founding erorr float: ",flattened_input[j][i+3], flattened_input[j],self.weights[i][j])
                sum_val += self.weights[i][j] * flattened_input[j]
            weighted_sum.append(sum_val + self.biases[i])
        #print("weighted_sum:", weighted_sum)
        return weighted_sum

    def backward(self, dL_dout, lr):
        #print("dl_dout:", dL_dout)
        dL_dy = self.calculate_loss_gradient(dL_dout)
        #print("dl_dy:", len(dL_dy), dL_dy)
        dL_dw = self.calculate_weight_gradient(dL_dy)
        dL_db = dL_dy
        dL_dinput = self.calculate_input_gradient(dL_dy)
        self.update_weights(lr, dL_dw)
        self.update_biases(lr, dL_db)
        return self.reshape_input_gradient(dL_dinput)

    def reshape_input_gradient(self, dL_dinput):
      #print("shape of input:", len(self.input_data), len(self.input_data[0]), len(self.input_data[0][0]))
      din = [[[0 for _ in range(len(self.input_data[0][0]))] for _ in range(len(self.input_data[0]))] for _ in range(len(self.input_data))]
      #print(dL_dinput)
      for i in range(len(self.input_data)):
        for j in range(len(self.input_data[0])):
          for c in range(len(self.input_data[0][0])):
            #print(j * len(self.input_data[0]) + c)
            din[i][j][c] = dL_dinput[i][j * len(self.input_data[0]) + c]
      return din


    def update_biases(self, lr, dl_db):
      #print("dldb:", dl_db)
      for i in range(len(self.biases)):
        self.biases[i] -= lr * dl_db[i]

    def update_weights(self, lr, dl_dw):
      for i in range(self.output_size):
        for j in range(self.input_size):
          #print(i,j)
          self.weights[i][j] -= lr * dl_dw[i][j]

    def calculate_loss_gradient(self, dL_dout):
        dL_dy = []
        dout2 = []
        for m in range(len(dL_dout)):
          for n in range(len(dL_dout[0])):
            dout2.append(dL_dout[m][n])

        sfmmatr = self.softmax_derivative(self.output)
        #print("softmax:", sfmmatr)
        #print("selfoutput:", len(self.output), self.output)
        #print("dL_dout:", len(dL_dout), dL_dout)
        for i in range(len(self.output)):
            sum_val = 0
            for j in range(len(self.output)):
                sum_val += sfmmatr[i][j] * dout2[j]
            dL_dy.append(sum_val)
        #print("softmax_dldy:", dL_dy)
        return dL_dy

    def calculate_weight_gradient(self, dL_dy):
        dL_dw = [[0 for _ in range(self.input_size)] for _ in range(len(dL_dy))]
        din = []
        for m in self.input_data:
          for n in m:
            for val in n:
              din.append(val)
        #print("len of dl_dy:", len(dL_dy))
        #print("len of inputsize:", len(din))
        for i in range(len(dL_dy)):
            for j in range(len(din)):
              dL_dw[i][j] = dL_dy[i] * din[j]
        return dL_dw

    def calculate_input_gradient(self, dL_dy):
        dL_dinput = [[0 for _ in range(self.input_size)] for _ in range(len(self.input_data))]
        for i in range(len(dL_dy)):
            for j in range(self.input_size):
                dL_dinput[i // self.output_size][j] += dL_dy[i] * self.weights[i][j]
        return dL_dinput

In [ ]:
def cross_entropy_loss(predictions, targets):
    num_samples = 10
    epsilon = 1e-7
    # Clip predictions to avoid numerical instability
    clipped_predictions = []
    for p in predictions:
        if p < epsilon:
            clipped_predictions.append(epsilon)
        elif p > 1 - epsilon:
            clipped_predictions.append(1 - epsilon)
        else:
            clipped_predictions.append(p)

    # Calculate the categorical cross-entropy loss
    loss = 0
    for i in range(num_samples):
        loss -= targets[i] * ln(clipped_predictions[i])

    loss /= num_samples

    return loss

def ln(x):
  n=1000
  return n*((x**(1/n))-1)
def cross_entropy_loss_gradient(actual_labels, predicted_probs):
    num_samples = actual_labels.shape[0]
    epsilon = 1e-7
    # Calculate the gradient of the cross-entropy loss function
    gradient = []
    for i in range(num_samples):
        gradient.append(-actual_labels[i] / (predicted_probs[i] + epsilon) / num_samples)

    return gradient

def train_network(X, y, conv, pool, full, lr=0.01, epochs=20):
    for epoch in range(epochs):
        print("epoch: ", epoch)
        total_loss = 0.0
        correct_predictions = 0
        for i in range(len(X)):
            # Forward pass
            conv_out = conv.forward(X[i])
            pool_out = pool.forward(conv_out)
            #print("pool_out:", pool_out)
            full_out = full.forward(pool_out)

            # Calculate loss and accuracy
            #print("fullout:", full_out)
            #print("yi:", y[i])
            loss = cross_entropy_loss(full_out, y[i])
            total_loss += loss
            #print("loss:", total_loss, loss)

            # Converting to One-Hot encoding
            one_hot_pred = [0] * len(full_out)
            one_hot_pred[max(range(len(full_out)), key=lambda x: full_out[x])] = 1
            #print("onehot:", one_hot_pred)
            #one_hot_pred = flatten_list(one_hot_pred)

            num_pred = max(range(len(one_hot_pred)), key=lambda x: one_hot_pred[x])
            num_y = max(range(len(y[i])), key=lambda x: y[i][x])

            if num_pred == num_y:
                correct_predictions += 1

            # Backward pass / passing the gradient of the loss function backwards
            gradient = cross_entropy_loss_gradient(y[i], full_out)
            #print("len of gradient: ",gradient)
            #print("len of gradient: ", len(gradient))
            #gradient = reshape_list(gradient, (-1, 1))
            j = 0
            new_grad = [[0 for _ in range(1)] for _ in range(len(gradient))]
            #print("new_grad:", new_grad)
            for i in range(len(gradient)):
              new_grad[i][j] = gradient[i]
            #print("new_grad:", new_grad)
            full_back = full.backward(new_grad, lr)
            pool_back = pool.backward(full_back, lr)
            conv_back = conv.backward(pool_back, lr)

        # Print epoch statistics
        average_loss = total_loss / len(X)
        accuracy = correct_predictions / len(X) * 100.0
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {average_loss:.4f} - Accuracy: {accuracy:.2f}%")

In [ ]:
def predict(input_sample, conv, pool, full):
    # Forward pass through convolutional and pooling layers
    conv_out = conv.forward(input_sample)
    pool_out = pool.forward(conv_out)

    # Flatten the output feature maps
    flattened_output = pool_out.flatten()

    # Forward pass through fully connected layer
    predictions = full.forward(flattened_output)

    return predictions

In [ ]:
import tensorflow.keras as keras
import numpy as np

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
X_train = train_images[:5000] / 255.0
y_train = train_labels[:5000]

X_test = train_images[5000:10000] / 255.0
y_test = train_labels[5000:10000]

X_train.shape

from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_test[0]

conv = Convolution(X_train[0].shape, 6, 1)
input_hi, input_wi = X_train[0].shape
print(input_hi-5)
pool = MaxPool(2, 1, input_hi-5, input_wi-5)
full = Fully_Connected(121, 10)

train_network(X_train, y_train, conv, pool, full)

predictions = []

for data in X_test:
    pred = predict(data, conv, pool, full)
    one_hot_pred = [0] * len(pred)
    one_hot_pred[max(range(len(pred)), key=lambda x: pred[x])] = 1
    predictions.append(flatten_list(one_hot_pred))

predictions = reshape_list(predictions, (-1, len(one_hot_pred)))


from sklearn.metrics import accuracy_score

accuracy_score(predictions, y_test)